In [1]:
%serialconnect to --port=COM3 --baud=115200
import time

Connecting to --port=COM3 --baud=115200 
Ready.


In [2]:
import machine


class accel():
    def __init__(self, i2c, addr=0x68):
        self.iic = i2c
        self.addr = addr
        self.iic.start()
        self.iic.writeto(self.addr, bytearray([107, 0]))
        self.iic.stop()

    def get_raw_values(self):
        self.iic.start()
        a = self.iic.readfrom_mem(self.addr, 0x3B, 14)
        self.iic.stop()
        return a

    def get_ints(self):
        b = self.get_raw_values()
        c = []
        for i in b:
            c.append(i)
        return c

    def bytes_toint(self, firstbyte, secondbyte):
        if not firstbyte & 0x80:
            return firstbyte << 8 | secondbyte
        return - (((firstbyte ^ 255) << 8) | (secondbyte ^ 255) + 1)

    def get_values(self):
        raw_ints = self.get_raw_values()
        vals = {}
        #vals["AcX"] = self.bytes_toint(raw_ints[0], raw_ints[1])
        #vals["AcY"] = self.bytes_toint(raw_ints[2], raw_ints[3])
        #vals["AcZ"] = self.bytes_toint(raw_ints[4], raw_ints[5])
        #vals["Tmp"] = self.bytes_toint(raw_ints[6], raw_ints[7]) / 340.00 + 36.53
        vals["GyX"] = self.bytes_toint(raw_ints[8], raw_ints[9])
        vals["GyY"] = self.bytes_toint(raw_ints[10], raw_ints[11])
        vals["GyZ"] = self.bytes_toint(raw_ints[12], raw_ints[13])
        return vals  # returned in range of Int16
        # -32768 to 32767

    def val_test(self):
        from time import sleep
        while 1:
            print(self.get_values())
            sleep(0.05)

In [3]:
from machine import SoftI2C
from machine import Pin
from machine import sleep
import _thread
#from IPython.display import clear_output
i2c1 = SoftI2C(scl=Pin(22), sda=Pin(21))       #initializing the I2C method for ESP8266
i2c2 = SoftI2C(scl=Pin(18), sda=Pin(19)) 
mpu1 = accel(i2c1)
mpu2 = accel(i2c2)
while True:
 mpu1.get_values()
 mpu2.get_values()
 #clear_output(wait=True)
 print("mpu1: ", mpu1.get_values())
 print("mpu2: ", mpu2.get_values())
 sleep(500)

mpu1:  {'GyZ': -15, 'GyY': -1082, 'GyX': 226}
mpu2:  {'GyZ': -1, 'GyY': -1, 'GyX': -1}
mpu1:  {'GyZ': -7, 'GyY': -1071, 'GyX': 189}


Traceback (most recent call last):
  File "<stdin>", line 15, in <module>
  File "<stdin>", line 31, in get_values
  File "<stdin>", line 14, in get_raw_values
OSError: [Errno 19] ENODEV
